In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2024-03-07T13:29:52.621041-05:00

Python implementation: CPython
Python version       : 3.7.11
IPython version      : 7.10.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 4.15.0-161-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 40
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: 
  GPU 0: Tesla V100-DGXS-32GB
  GPU 1: Tesla V100-DGXS-32GB
  GPU 2: Tesla V100-DGXS-32GB
  GPU 3: Tesla V100-DGXS-32GB



In [4]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import optuna
import gc
import logging

In [5]:
%watermark --iversions

dask   : 2022.2.0
logging: 0.5.1.2
optuna : 2.10.1
pandas : 1.3.5
numpy  : 1.21.6
xgboost: 1.6.2



In [6]:
!nvidia-smi -L

GPU 0: Tesla V100-DGXS-32GB (UUID: GPU-d752af09-1f62-bf3b-4f70-78b84e9e41f6)
GPU 1: Tesla V100-DGXS-32GB (UUID: GPU-054a4a35-f98a-3ebc-9100-0f697c246b43)
GPU 2: Tesla V100-DGXS-32GB (UUID: GPU-454525d4-bebd-7fb8-0ba3-3b85e2f99abd)
GPU 3: Tesla V100-DGXS-32GB (UUID: GPU-af0fb74e-f5eb-0833-17ff-494cc6bdcee1)


In [7]:
cluster = LocalCUDACluster(n_workers=4)
client = Client(cluster)

In [8]:
!nvidia-smi

Thu Mar  7 13:30:10 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                    0 |
| N/A   38C    P0    51W / 300W |    773MiB / 32505MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [9]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [10]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_fold_{i}_drop_shap_2.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}_drop_shap_2.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [11]:
train = pd.read_csv('../input/train.csv.zip')

target = train['target'].values
target

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    gini = gini_normalized(target, train_oof)
    
    return gini

In [13]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_7.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_porto_7.db", study_name="five_fold_optuna_xgb_7", direction='maximize')

In [14]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 77% CPU time recently (threshold: 10%)


CPU times: user 1min 11s, sys: 20.5 s, total: 1min 32s
Wall time: 15min 33s


In [15]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.261362,0.020140,0.384205,logloss,0.038935,0.070718,16,274,binary:logistic,0.416098,gpu_hist,COMPLETE
1,1,0.266650,0.073048,0.834241,logloss,0.298164,0.001174,24,211,binary:logistic,0.887860,gpu_hist,COMPLETE
2,2,0.268777,0.001554,0.426361,logloss,0.007277,0.001208,16,250,binary:logistic,0.871017,gpu_hist,COMPLETE


In [16]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_7.csv', index=False)
df.head(8)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 77% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 77% CPU time recently (thre

CPU times: user 1min 37s, sys: 18.9 s, total: 1min 56s
Wall time: 18min 29s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.261362,0.020140,0.384205,logloss,0.038935,0.070718,16,274,binary:logistic,0.416098,gpu_hist,COMPLETE
1,1,0.266650,0.073048,0.834241,logloss,0.298164,0.001174,24,211,binary:logistic,0.887860,gpu_hist,COMPLETE
2,2,0.268777,0.001554,0.426361,logloss,0.007277,0.001208,16,250,binary:logistic,0.871017,gpu_hist,COMPLETE
3,3,0.270365,6.638314,0.679528,logloss,0.005558,0.005153,12,175,binary:logistic,0.424908,gpu_hist,COMPLETE
4,4,0.255800,0.950049,0.880113,logloss,0.735849,0.001324,9,178,binary:logistic,0.736536,gpu_hist,COMPLETE
5,5,0.274685,2.638802,0.844755,logloss,0.004442,0.016047,14,68,binary:logistic,0.543025,gpu_hist,COMPLETE
6,6,0.276926,0.155249,0.892546,logloss,0.195688,0.005227,24,227,binary:logistic,0.897245,gpu_hist,COMPLETE
7,7,0.247681,0.232662,0.686948,logloss,1.354195,0.047604,22,169,binary:logistic,0.894568,gpu_hist,COMPLETE


In [17]:
study.best_trial.params

{'alpha': 0.15524863803702552,
 'colsample_bytree': 0.892546250706431,
 'eval_metric': 'logloss',
 'lambda': 0.19568781717035563,
 'learning_rate': 0.005227189786595764,
 'max_depth': 24,
 'min_child_weight': 227,
 'objective': 'binary:logistic',
 'subsample': 0.8972448166220794,
 'tree_method': 'gpu_hist'}

In [18]:
df.value.max()

0.27692596618509957

In [19]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_7.csv', index=False)
df.head(20)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 79% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 79% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 79% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 79% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 79% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 79% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 80% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 80% CPU time recently (thre

CPU times: user 24min 44s, sys: 4min 16s, total: 29min
Wall time: 3h 45min 57s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.261362,0.020140,0.384205,logloss,0.038935,0.070718,16,274,binary:logistic,0.416098,gpu_hist,COMPLETE
1,1,0.266650,0.073048,0.834241,logloss,0.298164,0.001174,24,211,binary:logistic,0.887860,gpu_hist,COMPLETE
2,2,0.268777,0.001554,0.426361,logloss,0.007277,0.001208,16,250,binary:logistic,0.871017,gpu_hist,COMPLETE
3,3,0.270365,6.638314,0.679528,logloss,0.005558,0.005153,12,175,binary:logistic,0.424908,gpu_hist,COMPLETE
4,4,0.255800,0.950049,0.880113,logloss,0.735849,0.001324,9,178,binary:logistic,0.736536,gpu_hist,COMPLETE
5,5,0.274685,2.638802,0.844755,logloss,0.004442,0.016047,14,68,binary:logistic,0.543025,gpu_hist,COMPLETE
6,6,0.276926,0.155249,0.892546,logloss,0.195688,0.005227,24,227,binary:logistic,0.897245,gpu_hist,COMPLETE
7,7,0.247681,0.232662,0.686948,logloss,1.354195,0.047604,22,169,binary:logistic,0.894568,gpu_hist,COMPLETE
8,8,0.241843,0.460575,0.727188,logloss,0.071139,0.044507,12,100,binary:logistic,0.808134,gpu_hist,COMPLETE
9,9,0.238010,0.104018,0.529525,logloss,0.008265,0.074132,12,161,binary:logistic,0.677691,gpu_hist,COMPLETE


In [20]:
df.value.max()

0.2834580736058561

In [21]:
study.best_trial.params

{'alpha': 5.966346651035019,
 'colsample_bytree': 0.9315963658349328,
 'eval_metric': 'logloss',
 'lambda': 1.3055969464237063,
 'learning_rate': 0.01210408710226175,
 'max_depth': 7,
 'min_child_weight': 160,
 'objective': 'binary:logistic',
 'subsample': 0.9842765843371922,
 'tree_method': 'gpu_hist'}